# Aprendizagem por reforço: jogo da velha

In [9]:
%matplotlib tk

from owg_board import owg
from owg_players import jb, miope, cientista, cientista_sovina, cientista_cauteloso, cientista_conciliador, epsilon_edson
from tqdm import tqdm

Inicializa jogadores

In [15]:
p1 = epsilon_edson(desconto = 0.9, epsilon = 0.5, nome = 'epsilon_edson_1MM')

p2 = epsilon_edson(desconto = 0.9, epsilon = 0.5)

Reseta o tabuleiro (desnecessário no primeiro round)

In [16]:
p1.reset()
p2.reset()

### Treino por self-learning

p1 joga contra p2 por $n$ jogos

In [17]:
# Número de jogos para treinar
n = 1000

In [18]:
# Contadores
vitorias = 0
derrotas = 0
empates = 0

# Proporções de cada resultado ao longo do tempo
pempate = []
pjog1 = []
pjog2 = []

for i in tqdm(range(n)):
    
    # Alterna quem começa o jogo
    if i % 2 == 0:
        # Joga p1
        movimento = p1.joga()
        # Comunica o movimento para p2
        p2.comunica(movimento)
        # Checa se o jogo acabou
        res, _ = p2.board.check_result()
        while res is None:
            # Enquanto não acabar
            movimento = p2.joga()
            if movimento is not None:
                p1.comunica(movimento)
                movimento = p1.joga()           
                if movimento is not None:
                    p2.comunica(movimento)
            res, _ = p2.board.check_result()
        
        # Checa o resultado e atualiza os contadores
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
    else:
        # Joga p2
        movimento = p2.joga()
        # Comunica o movimento para p1
        p1.comunica(movimento)
        # Checa se o jogo acabou
        res, _ = p1.board.check_result()
        while res is None:
            # Enquanto não acabar
            movimento = p1.joga()
            if movimento is not None:
                p2.comunica(movimento)
                movimento = p2.joga()          
                if movimento is not None:
                    p1.comunica(movimento)
            res, _ = p1.board.check_result()
        
        # Checa o resultado final e atualiza contadores
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
        
    pempate.append(empates / (i+1))
    pjog1.append(vitorias / (i+1))
    pjog2.append(derrotas / (i+1))

100%|██████████| 1000/1000 [00:01<00:00, 636.80it/s]


Gráfico da evoluçao das proporções de empate e vitória de cada jogador

In [20]:
from matplotlib import pyplot as plt
plt.figure(figsize = (20, 10))
plt.plot(pjog1, '-', label = p1.nome)
plt.plot(pempate, '-', label = 'Empate')
plt.plot(pjog2, '-', label = p2.nome)
plt.ylim([0,1])
plt.hlines(y = 0.5, xmin = 0, xmax = len(pjog1) , linestyles='dashed')
plt.grid(True)
plt.legend()
plt.show()

### Carregando os jogadores pré-treinados

In [21]:
import pickle
with open('cientista_1MM.pkl', 'rb') as arq:
    p1 = pickle.load(arq)
with open('cientista_cauteloso_1MM.pkl', 'rb') as arq:
    p2 = pickle.load(arq)
with open('epsilon_edson_1MM.pkl', 'rb') as arq:
    p3 = pickle.load(arq)    

### Analisando aprendizagem do jogador

Clique com o botão direito do mouse para exibir as ações possíveis e o valor estimado de cada uma delas de acordo com o jogador passado como argumento para a função

In [23]:
tab = owg()
tab.start_free(p2)

### Jogar contra

In [ ]:
# Jogar contra quem?
p = p1
p.reset()
tab = owg()
tab.start(p)